This notebook will be mainly used for the capstone project - IBM Data Science Professional Certificate

In [54]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from sklearn.cluster import KMeans

Using pandas web scrapper to get the table from HTML and store it into dataframe

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs = pd.read_html(url)

We need to get just the first table, so specifying index at 0 will give us right table

In [3]:
df = dfs[0]
df.head

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Removing not assigned boroughs

In [4]:
df = df[df['Borough'] != "Not assigned"]
df.reset_index(drop=True,inplace=True)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Check if Neighbourhood has cells with Not assigned value

In [5]:
df[df['Neighbourhood'] == "Not assigned"]

,Postal Code,Borough,Neighbourhood


Shape of the dataframe

In [6]:
df.shape

(103, 3)

In [7]:
from geopy.geocoders import Nominatim
import folium

In [8]:
geolocator = Nominatim(user_agent = "canada")

In [9]:
location = geolocator.geocode("Canada Post Gateway Processing Centre")

In [10]:
location is None

True

Iterate through neighborhoods to find longitude and lattitude. If neighborhood has more than one value, first will be used. Sometimes location will be None, in that case we will drop that row from dataframe

In [11]:
lg = []
lt = []
for index, row in df.iterrows():
    i = 0
    nb = row["Neighbourhood"].split(', ')
    location = geolocator.geocode(nb[i])
    if(location is None):
        df.drop([index],inplace=True)
        continue
    latitude = location.latitude
    longitude = location.longitude
    lg.append(longitude)
    lt.append(latitude)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
slt = pd.Series(lt)
slg = pd.Series(lg)

Ignore the warning. Adding latitude and longitude to df

In [13]:
df['Latitude'] = slt.values
df['Longitude'] = slg.values

<ipython-input-13-919dfefd0eaa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Latitude'] = slt.values
<ipython-input-13-919dfefd0eaa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Longitude'] = slg.values


In [14]:
df.reset_index(drop=True,inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,26.565264,-81.881723
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360457
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",51.533873,-0.205496


In [15]:
CLIENT_ID = '0EFYTJHRD1DOAWCZKIQ1PRDVWS43QMWZBMUN0ZOIU4ITC4OB' # your Foursquare ID
CLIENT_SECRET = 'D1G4IY5BOWOGAXEILTVQMB0M35EVYA1T5AH42HMGKYDTIEOI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0EFYTJHRD1DOAWCZKIQ1PRDVWS43QMWZBMUN0ZOIU4ITC4OB
CLIENT_SECRET:D1G4IY5BOWOGAXEILTVQMB0M35EVYA1T5AH42HMGKYDTIEOI


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
canada_venues = getNearbyVenues(df['Neighbourhood'], df['Latitude'], df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [40]:
canada_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,26.565264,-81.881723,Famous Tattoos,26.563877,-81.878446,Cosmetics Shop
1,Parkwoods,26.565264,-81.881723,Chappell Howard E Attorney At Law,26.566019,-81.886460,Lawyer
2,Victoria Village,43.732658,-79.311189,Jatujak,43.736208,-79.307668,Thai Restaurant
3,Victoria Village,43.732658,-79.311189,Armenian Kitchen,43.731071,-79.305390,Middle Eastern Restaurant
4,Victoria Village,43.732658,-79.311189,TTC Bus #24 Victoria Park Ave,43.735197,-79.306977,Bus Line


In [41]:
canada_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,1,1,1,1,1,1
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,11,11,11,11,11,11
"Bedford Park, Lawrence Manor East",12,12,12,12,12,12
...,...,...,...,...,...,...
"Willowdale, Willowdale West",47,47,47,47,47,47
Woburn,29,29,29,29,29,29
Woodbine Heights,9,9,9,9,9,9


In [44]:
canada_onehot = pd.get_dummies(canada_venues[['Venue Category']], prefix="", prefix_sep="")
canada_onehot['Neighborhood'] = canada_venues['Neighborhood']
fixed_columns = [canada_onehot.columns[-1]] + list(canada_onehot.columns[:-1])
canada_onehot = canada_onehot[fixed_columns]
canada_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
canada_grouped = canada_onehot.groupby('Neighborhood').mean().reset_index()
canada_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.052632,0.0,0.0,0.0
3,Bayview Village,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,"Willowdale, Willowdale West",0.021277,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
87,Woburn,0.000000,0.000000,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,...,0.034483,0.0,0.0,0.034483,0.0,0.0,0.000000,0.0,0.0,0.0
88,Woodbine Heights,0.000000,0.111111,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
89,York Mills West,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [47]:
num_top_venues = 5

for hood in canada_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = canada_grouped[canada_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                     venue  freq
0       Miscellaneous Shop   1.0
1              Opera House   0.0
2             Noodle House   0.0
3                Nightclub   0.0
4  New American Restaurant   0.0


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1  Sandwich Place  0.17
2             Pub  0.17
3     Coffee Shop  0.17
4             Gym  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
                  venue  freq
0           Supermarket  0.16
1                  Café  0.16
2         Shopping Mall  0.11
3  Fast Food Restaurant  0.11
4             Irish Pub  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.18
1   Persian Restaurant  0.09
2  Sporting Goods Shop  0.09
3          Gas Station  0.09
4          Fish Market  0.09


----Bedford Park, Lawrence Manor East----
                venue  freq
0        Soccer Field  0.17
1        Intersection  0.08
2  Mexican Restaurant  0.08
3          Sp

4               Asian Restaurant  0.04


----Northwood Park, York University----
                       venue  freq
0                       Park  0.50
1  Middle Eastern Restaurant  0.25
2             Baseball Field  0.25
3                     Office  0.00
4               Noodle House  0.00


----Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East----
           venue  freq
0           Park  0.38
1            Spa  0.12
2          River  0.12
3    Event Space  0.12
4  Metro Station  0.12


----Parkdale, Roncesvalles----
                     venue  freq
0              Post Office   1.0
1              Opera House   0.0
2             Noodle House   0.0
3                Nightclub   0.0
4  New American Restaurant   0.0


----Parkwoods----
            venue  freq
0  Cosmetics Shop   0.5
1          Lawyer   0.5
2     Yoga Studio   0.0
3          Museum   0.0
4    Noodle House   0.0


----Queen's Park, Ontario Pro

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = canada_grouped['Neighborhood']

for ind in np.arange(canada_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(canada_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Miscellaneous Shop,Women's Store,Field,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pub,Coffee Shop,Sandwich Place,Gym,Women's Store,Event Space,Eastern European Restaurant,Electronics Store,English Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Café,Supermarket,Fast Food Restaurant,Shopping Mall,Liquor Store,Sandwich Place,Clothing Store,Restaurant,Irish Pub,Brewery
3,Bayview Village,Bank,Pizza Place,Outdoor Supply Store,Fish Market,Sporting Goods Shop,Fast Food Restaurant,Gas Station,Sandwich Place,Breakfast Spot,Persian Restaurant
4,"Bedford Park, Lawrence Manor East",Soccer Field,Park,Soccer Stadium,Mexican Restaurant,Sports Bar,Dive Bar,Restaurant,Intersection,Racetrack,Convenience Store


In [71]:
# set number of clusters
kclusters = 5

canada_grouped_clustering = canada_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(canada_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [72]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

canada_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
canada_merged = canada_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

canada_merged.head() # check the last columns!

KeyError: 'Neighborhood'